In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import random
import os
import requests
from bs4 import BeautifulSoup, Comment
from datetime import datetime
import html5lib
import lxml

from utils import mp_to_minutes

In [2]:
# extract draft class from the url
def load_draft_class(draft_year):
    url = f"https://www.basketball-reference.com/draft/NBA_{draft_year}.html"

    df = pd.read_html(url)[0]
    df_links = pd.read_html(url, extract_links="body")[0]

    df.columns = [f"{a}_{b}".strip("_") for a, b in df.columns]
    df_links.columns = df.columns

    df = df[df["Round 1_Player"].notna()].copy()

    df["player"] = df["Round 1_Player"]
    df["player_id"] = df_links["Round 1_Player"].apply(
        lambda x: x[1].split("/")[-1].replace(".html", "")
        if isinstance(x, tuple) and x[1]
        else None
    )

    # 🔑 drop header rows like "Round 2"
    df = df[df["player_id"].notna()].copy()

    df["draft_year"] = draft_year
    df["rookie_season"] = draft_year + 1

    return df[["draft_year", "player", "player_id", "rookie_season"]]



In [3]:
# load or scrape draft class with caching
# def load_or_scrape_draft(year):
#     path = f"assets/drafts/draft_{year}.csv"
#     if os.path.exists(path):
#         return pd.read_csv(path)
    
#     df = load_draft_class(year)
#     df.to_csv(path, index=False)
#     return df

# function to scrape a single draft class
def scrape_and_save_year(year):
    df = load_draft_class(year)
    df.to_csv(f"assets/drafts/draft_{year}.csv", index=False)
    print(f"✓ saved {year}")


In [4]:
# iterate through years and save each draft class

# years = [2022, 2023, 2024, 2025]

# for y in years:
#     scrape_and_save_year(y)
#     time.sleep(10)

In [5]:
# freeze the draft years collected
import glob
draft_classes = pd.concat(
    [pd.read_csv(f) for f in glob.glob("assets/drafts/draft_20*.csv")],
    ignore_index=True
)
draft_classes.head(100)


,draft_year,player,player_id,rookie_season
0,2000,Kenyon Martin,martike01,2001
1,2000,Stromile Swift,swiftst01,2001
2,2000,Darius Miles,milesda01,2001
3,2000,Marcus Fizer,fizerma01,2001
4,2000,Mike Miller,millemi01,2001
...,...,...,...,...
95,2001,Michael Wright,wrighmi01,2002
96,2001,Earl Watson,watsoea01,2002
97,2001,Jamison Brewer,breweja01,2002
98,2001,Bobby Simmons,simmobo01,2002


In [6]:
# override rookie seasons for specific players
rookie_overrides = {
    "griffbl01": 2011,  # drafted 2009, rookie season 2010–11
    "embiijo01": 2016,  # drafted 2014, rookie season 2015-16
}

In [7]:
df = pd.read_html(
    "https://www.basketball-reference.com/players/j/jamesle01/gamelog/2004",
    attrs={"id": "player_game_log_reg"}
)[0]

print(list(df.columns))

['Rk', 'Gcar', 'Gtm', 'Date', 'Team', 'Unnamed: 5', 'Opp', 'Result', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']


In [8]:
def load_rookie_gamelog(player_id, season):
    first_letter = player_id[0]
    url = (
        f"https://www.basketball-reference.com/players/"
        f"{first_letter}/{player_id}/gamelog/{season}"
    )

    df = pd.read_html(
        url,
        attrs={"id": "player_game_log_reg"}
    )[0]

    # 1️⃣ KEEP ONLY REAL GAMES (must have an opponent)
    df = df[df["Opp"].notna()].copy()

    # 2️⃣ Convert MP to numeric (DNPs become NaN)
    df["MP"] = df["MP"].apply(mp_to_minutes)

    # 3️⃣ Drop games not played
    df = df[df["MP"].notna()].copy()

    return df


In [9]:
df = load_rookie_gamelog("jamesle01", 2004)

print("Rows:", len(df))
print(df[["Date", "Opp", "MP", "PTS"]].head(10))


Rows: 79
          Date  Opp         MP PTS
0   2003-10-29  SAC  42.833333  25
1   2003-10-30  PHO  40.350000  21
3   2003-11-01  POR  39.166667   8
4   2003-11-05  DEN  41.100000   7
5   2003-11-07  IND  43.733333  23
6   2003-11-08  WAS  44.500000  17
7   2003-11-10  NYK  33.650000  17
8   2003-11-12  MIA  42.666667  18
9   2003-11-14  BOS  35.600000  10
10  2003-11-15  PHI  46.950000  22


In [10]:
def aggregate_capped_minutes(
    gamelog,
    cap_minutes=900,
    stat_cols=("PTS", "TRB", "AST"),
):
    """
    Aggregate rookie production up to a capped number of minutes.
    """

    # Ensure counting stats are numeric
    gamelog = gamelog.copy()
    for stat in stat_cols:
        gamelog[stat] = pd.to_numeric(gamelog[stat], errors="coerce")

    total_minutes = 0.0
    totals = {stat: 0.0 for stat in stat_cols}

    for _, row in gamelog.iterrows():
        mp = row["MP"]

        if total_minutes >= cap_minutes:
            break

        remaining = cap_minutes - total_minutes

        if mp <= remaining:
            weight = 1.0
            used_mp = mp
        else:
            weight = remaining / mp
            used_mp = remaining

        total_minutes += used_mp

        for stat in stat_cols:
            if pd.notna(row[stat]):
                totals[stat] += row[stat] * weight

    return {
        "minutes_used": total_minutes,
        **totals,
    }

In [11]:
lebron_2004 = load_rookie_gamelog("jamesle01", 2004)

agg_300 = aggregate_capped_minutes(lebron_2004, cap_minutes=300)
agg_900 = aggregate_capped_minutes(lebron_2004, cap_minutes=900)

agg_300, agg_900

({'minutes_used': 300.0,
  'PTS': 124.18750000000001,
  'TRB': 52.03125,
  'AST': 48.40625},
 {'minutes_used': 900.0,
  'PTS': 386.99963302752303,
  'TRB': 146.3783486238532,
  'AST': 140.83779816513763})

In [12]:
# process Kevin Durant's rookie season
kd_2008 = load_rookie_gamelog("duranke01", 2008)
agg_kd_900 = aggregate_capped_minutes(kd_2008, cap_minutes=900)
agg_kd_900

{'minutes_used': 900.0,
 'PTS': 536.1828947368422,
 'TRB': 115.93947368421054,
 'AST': 54.9092105263158}

In [13]:
# process derrick rose's rookie season
dr_2008 = load_rookie_gamelog("rosede01", 2009)
agg_dr_900 = aggregate_capped_minutes(dr_2008, cap_minutes=900)
agg_dr_900

{'minutes_used': 900.0,
 'PTS': 420.29240596167494,
 'TRB': 91.07700496806245,
 'AST': 146.50780695528744}

In [14]:
# process jayson tatum's rookie season
jt_2017 = load_rookie_gamelog("tatumja01", 2018)
agg_jt_900 = aggregate_capped_minutes(jt_2017, cap_minutes=900)
agg_jt_900

{'minutes_used': 900.0,
 'PTS': 406.7432835820894,
 'TRB': 166.79820895522383,
 'AST': 37.89910447761192}

In [15]:
# process karl-anthony towns's rookie season
kt_2014 = load_rookie_gamelog("townska01", 2016)
agg_kt_900 = aggregate_capped_minutes(kt_2014, cap_minutes=900)
agg_kt_900

{'minutes_used': 900.0,
 'PTS': 499.53939714436797,
 'TRB': 296.2157588577472,
 'AST': 32.22157588577472}

In [16]:
# calculate rate stats
def add_rate_stats(agg, minutes_base=36):
    factor = minutes_base / agg["minutes_used"]
    return {
        f"{k}_per_{minutes_base}": v * factor
        for k, v in agg.items()
        if k not in ("minutes_used",)
    }

In [18]:
# batch process rookie classes
cap = 900
results = []
errors = []

for _, row in draft_classes.iterrows():
    try:
        gamelog = load_rookie_gamelog(row.player_id, row.rookie_season)
        agg = aggregate_capped_minutes(gamelog, cap_minutes=cap)

        results.append({
            **row.to_dict(),
            **agg
        })

    except Exception as e:
        errors.append({
            "player_id": row.player_id,
            "rookie_season": row.rookie_season,
            "error": str(e),
        })

rookie_caps_900 = pd.DataFrame(results)
errors_df = pd.DataFrame(errors)

In [23]:
rookie_caps_900.head(200)

,draft_year,player,player_id,rookie_season,minutes_used,PTS,TRB,AST
0,2000,Kenyon Martin,martike01,2001,900.000000,287.515991,197.842217,46.000000
1,2000,Stromile Swift,swiftst01,2001,900.000000,276.023829,186.755957,17.000000
2,2000,Darius Miles,milesda01,2001,900.000000,316.055385,227.523077,46.513846
3,2000,Marcus Fizer,fizerma01,2001,900.000000,354.029369,188.726872,47.000000
4,2000,Mike Miller,millemi01,2001,900.000000,328.808327,154.819095,56.000000
5,2000,DerMarr Johnson,johnsde03,2001,900.000000,291.886534,113.591022,48.000000
6,2000,Chris Mihm,mihmch01,2001,900.000000,339.696023,215.278409,10.379735
7,2000,Jamal Crawford,crawfja01,2001,900.000000,240.824859,70.329944,124.164972
8,2000,Joel Przybilla,przybjo01,2001,267.066667,27.000000,71.000000,2.000000
9,2000,Keyon Dooling,doolike01,2001,900.000000,321.989399,61.123675,128.123675


In [22]:
errors_df.head(200)

,player_id,rookie_season,error
0,thomaet01,2001,No tables found
1,alexaco02,2001,HTTP Error 429: Too Many Requests
2,cleavma01,2001,HTTP Error 429: Too Many Requests
3,collija02,2001,HTTP Error 429: Too Many Requests
4,turkohe01,2001,HTTP Error 429: Too Many Requests
...,...,...,...
195,szewcsz01,2004,HTTP Error 429: Too Many Requests
196,austima01,2004,HTTP Error 429: Too Many Requests
197,hansetr01,2004,HTTP Error 429: Too Many Requests
198,blakest01,2004,HTTP Error 429: Too Many Requests


In [25]:
retry_ids = errors_df.query("error.str.contains('429')", engine="python")
# rerun only these later, slowly
retry_ids

,player_id,rookie_season,error
1,alexaco02,2001,HTTP Error 429: Too Many Requests
2,cleavma01,2001,HTTP Error 429: Too Many Requests
3,collija02,2001,HTTP Error 429: Too Many Requests
4,turkohe01,2001,HTTP Error 429: Too Many Requests
5,masonde01,2001,HTTP Error 429: Too Many Requests
...,...,...,...
1524,olbrila01,2026,HTTP Error 429: Too Many Requests
1525,richawi02,2026,HTTP Error 429: Too Many Requests
1526,shulgma01,2026,HTTP Error 429: Too Many Requests
1527,niangsa01,2026,HTTP Error 429: Too Many Requests


In [26]:
rookie_caps_900["hit_cap"] = rookie_caps_900["minutes_used"] >= cap